<a href="https://colab.research.google.com/github/VSSARATHI/Video-Captioning-Using-Attention-based-Bi-directional-LSTM/blob/master/tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataframe = pd.read_csv("/content/drive/My Drive/Colab Notebooks/CV_Project/data.csv", encoding= 'unicode_escape')
dataframe.head()

,VideoID,Start,End,WorkerID,Source,AnnotationTime,Language,Description
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mv89psg6zh4,33.0,46.0,588702.0,unverified,55.0,Slovene,Papagaj se umiva pod tekoo vodo v lijaku.
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mv89psg6zh4,33.0,46.0,588702.0,unverified,37.0,Slovene,Papagaj se umiva pod tekoo vodo v lijaku.
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
d = dataframe[dataframe['Language']=='English']
d.head()

,VideoID,Start,End,WorkerID,Source,AnnotationTime,Language,Description
37,mv89psg6zh4,33.0,46.0,682611.0,clean,66.0,English,A bird in a sink keeps getting under the runni...
39,mv89psg6zh4,33.0,46.0,760882.0,clean,16.0,English,A bird is bathing in a sink.
41,mv89psg6zh4,33.0,46.0,878566.0,clean,76.0,English,A bird is splashing around under a running fau...
43,mv89psg6zh4,33.0,46.0,707318.0,clean,14.0,English,A bird is bathing in a sink.
45,mv89psg6zh4,33.0,46.0,135621.0,clean,58.0,English,A bird is standing in a sink drinking water th...


In [ ]:
d=d.loc[:,['VideoID','Description']]

In [ ]:
d1=d.values.tolist()

In [ ]:
tokens = np.matrix(d1,dtype=np.str)

In [ ]:
tokens

matrix([['mv89psg6zh4',
         'A bird in a sink keeps getting under the running water from a faucet.'],
        ['mv89psg6zh4', 'A bird is bathing in a sink.'],
        ['mv89psg6zh4',
         'A bird is splashing around under a running faucet.'],
        ...,
        ['m7x8uIdg2XU', 'The lady added a cream sauce to the pasta.'],
        ['m7x8uIdg2XU', 'women are cooking her kichen'],
        ['m7x8uIdg2XU', 'The woman is pouring cream over the pasta.']],
       dtype='<U742')

In [ ]:
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [ ]:
import tensorflow as tf
import unicodedata
import re
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [ ]:
for i in range(tokens.shape[0]):
    tokens[i,1] = preprocess_sentence(tokens[i,1])
tokens[24087,:]

matrix([['Kxa0mnDj0bs',
         '<start> a big cat fought with a smll black cat . tat big cat is drinking its food . it also dashes the little child . somebody helping it for iching its body . small black cat act as dead one . the cat jumps higher . two cats are disturbing a person . one cat fell into a box . a cat gets scarred . a cat forced down a child in ground . a cat disturbes the fishes and fish tank settings . a dog and a cat fights each other . a bear scarred of a cat . a rat rotates a cat . a cat rotates a bed . on friday work , penguins enjoys with its family and swims in water . on monday work , a single polar fell in ice . a dog was skateboarding . eventhough a drum falls , its getting skate board safely and again doing excellent skateboarding in a roa']],
       dtype='<U742')

In [ ]:
lines = tokens[:,1]

In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    return tensor, lang_tokenizer

In [ ]:
z = []
for i in range(len(lines)):
    z.append(lines[i,0])
# z
z.pop(24087)
for i in range(len(z)):
    if len(z[i].split())==161:
        print(i,z[i])
print(z[0])

<start> a bird in a sink keeps getting under the running water from a faucet . <end>


In [ ]:
input_tensor, inp_lang_tokenizer = tokenize(z)

def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

convert(inp_lang_tokenizer, input_tensor[0])

2 ----> <start>
1 ----> a
252 ----> bird
9 ----> in
1 ----> a
467 ----> sink
2364 ----> keeps
245 ----> getting
317 ----> under
6 ----> the
47 ----> running
32 ----> water
36 ----> from
1 ----> a
900 ----> faucet
4 ----> .
3 ----> <end>


In [ ]:
import os
files = []
path = '/content/drive/My Drive/Colab Notebooks/CV_Project/YouTubeClips'
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.avi' in file:
            files.append(os.path.join(r, file))

In [ ]:
len(f)

1993

In [ ]:
import tensorflow as tf
input_shape=(224,224,3)
model_vgg = tf.keras.applications.VGG16(include_top=True,input_shape=input_shape)

new_input = model_vgg.input
hidden_layer = model_vgg.layers[-2].output

base_model = tf.keras.Model(new_input, hidden_layer)
base_model.summary()

553467904/553467096 [==============================] - 33s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm
def extract_video_features(path, N, model):
    video = np.zeros(shape=(N,224,224,3),dtype=np.float64)
    vidcap = cv2.VideoCapture(path)
    total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    if total_frames >= N : 
        frames_step = total_frames//N
        for i in range(N):
            vidcap.set(1,i*frames_step)
            success,image = vidcap.read()
            image = cv2.resize(image,(224,224))
            video[i] = image
    else:
        j = int(N-total_frames)
        for i in range(int(total_frames)):
            vidcap.set(1,i)
            success,image = vidcap.read()
            image = cv2.resize(image,(224,224))
            video[i+j] = image
    vidcap.release()     
    features = model(video)
    return np.matrix(features)

In [ ]:
from tqdm import tqdm
videos = []
description = []
features = []
a = {}
count = 0
for i in tqdm(range(len(tokens))):
    if count>=10000:
        break
    if type(a.get(tokens[:,0][i].item())) == type(None) :
        for j in range(len(f)):
            if tokens[:,0][i].item() in f[j]:
                e = extract_video_features(files[j], 30, base_model)
                features.append(e)
                description.append(tokens[:,1][i].item())
                a[tokens[:,0][i].item()] = e
                videos.append(tokens[:,0][i].item())
                count+=1
    else :
        features.append(a[tokens[:,0][i].item()])
        description.append(tokens[:,1][i].item())
        videos.append(tokens[:,0][i].item())
        count+=1

In [ ]:
np.save("/content/drive/My Drive/Colab Notebooks/CV_Project/x_train1",features)

In [ ]:
np.save("/content/drive/My Drive/Colab Notebooks/CV_Project/y_train1",description)

In [ ]:
np.save("/content/drive/My Drive/Colab Notebooks/CV_Project/videos1",videos)